# Prediction Model Diarrhoea Version 2
Last Big Modification: 17/10/2022
Description: 3 Models CNN, LSTM, LSTM-ATT

In [1]:
!nvidia-smi

Sat Feb 11 15:19:02 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.60.13    Driver Version: 525.60.13    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 6000     On   | 00000000:21:00.0 Off |                  Off |
| 34%   49C    P2   108W / 260W |   3207MiB / 24576MiB |     99%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# Install packages
!pip install -U scikit-learn
!pip install ftfy
# !pip install xlsxwriter 
# !pip install xlwt

In [3]:
# Imports
import tensorflow as tf 
import os

from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.layers import Dense

import traceback
import sys
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable

from ftfy import fix_text
import warnings
warnings.simplefilter("ignore", UserWarning)

plt.rcParams['figure.figsize'] = [7, 3]

# Input path

In [4]:
prj_path = '/home/mlworker/Quang/HealthCare/Source_14012023_v4/'
prj_path_opt= prj_path + "optimize_hyperparam/opt_results/opt_results_10022023_v4/"
data_path = prj_path + "data/new_data/DH/squeezed/"

# result path 
path = prj_path + "results/results_11022023/"
path_metrix = path + 'metrix/'
path_all_mix_cnn = path + 'all_mix_data/cnn/'
path_all_mix_lstm = path + 'all_mix_data/lstm/'
path_all_mix_lstm_att = path + 'all_mix_data/lstm_att/'


os.chdir(prj_path)

# Class Train and Support Functions for Preprocess

---



In [5]:
def seed_everything(seed: int):
    import random, os
    import numpy as np
    import torch
    
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

seed_everything(42)

In [6]:
# Set hyperparameters as args using the Configuration class
class Configuration():
    def __init__(self):
      # lấy bộ test dài 24 tháng = 2 năm
        self.test_size = 36
        # là nhìn vào dữ liệu trước 3 tháng và dự phóng        
        self.look_back = 3
        # dự phóng 3 tháng
        self.n_predicted_month = 3
        self.n_features = 3
        self.seed = 42
        # mỗi phần tử x trong tập suppervise có độ lớn là 16 = 16 tháng
        self.batch_size = 16
        self.device = torch.device("cuda")
        self.epochs = 300

args = Configuration()

In [7]:
# Define data (pre-)processing functions
# modification
def get_city_data(city_name):
    """Returns Diarrhoea rate and climate data only""" 
    city_data = dict_full_data[city_name].drop(columns=['Diarrhoea_cases', 'province', 'year_month',
                                                        'Influenza_rates','Dengue_fever_rates',
                                                        'Influenza_cases','Dengue_fever_cases', 'year', 'month',
                                                        'population_male','population_female','population_urban',
                                                        'population_countryside','area','population_density',
                                                        'population_average','birth_rate','urban_water_usage_rate',
                                                        'clean_water_rate_all','poverty_rate','toilet_rate'
                                                        ], 
                                                                  axis=1, 
                                                                  inplace=False)    
    return city_data

def convert_to_stationary(city_data):
    """Subtracts previous value for all cols except disease rates"""
    for col_name in city_data.columns:
        if col_name != 'Diarrhoea_rates':
            try:
                city_data[col_name] = city_data[col_name] - city_data[col_name].shift()
            except:
                print(col_name)
    return city_data

def impute_missing_value(city_data):
    """
    Cơ bản dữ liệu bị thiếu sót rất nhiều: Như Điện Biên 1997 -2003 là thiếu dữ liệu về bệnh
    Hàm này sẽ tự sinh ra dữ liệu bị thiếu. Nếu tháng nào không có số liệu thì tính như sau:
    12 tháng đầu không có số liệu thì gán = 0
    tháng 13-24 không có số liệu, sẽ lấy giá trị của tháng cùng kỳ năm trước
    tháng từ 24 trở đi sẽ lấy giá trị nhỏ nhất của 2 tháng cùng kỳ trong 2 năm gần nhất.
    Do Điện Biên bằng 0 nên sau khi xử lý từ 1997 -2003 là đều = 0.  
    """
    for col in city_data.columns:
        for index in range(len(city_data[col])):
            if np.isnan(city_data[col].iloc[index]):
                if index < 12:
                    city_data[col].iloc[index] = 0
                elif index >= 12 and index <= 24:
                    city_data[col].iloc[index] = city_data[col].iloc[index - 12]
                else:
                    city_data[col].iloc[index] = min(city_data[col].iloc[index - 12], city_data[col].iloc[index - 24])
    return city_data

In [8]:
cities = [
    'An Giang', 'BR Vũng Tàu', 'Bình Phước', 'Bình Thuận', 'Bình Định',
        'Bạc Liêu', 'Bắc Kạn', 'Bắc Giang', 'Cao Bằng', 'Cà Mau',
        'Cần Thơ', 'Gia Lai', 'Hà Giang', 'Hà Nội', 'Hà Tĩnh', 'Hòa Bình',
        'Hưng Yên', 'Hải Dương', 'Hải Phòng', 'Khánh Hòa', 'Kiên Giang',
        'Kon Tum', 'Lai Châu', 'Long An', 'Lào Cai', 'Lâm Đồng',
        'Lạng Sơn', 'Nam Định', 'Nghệ An', 'Ninh Bình', 'Ninh Thuận',
        'Phú Thọ', 'Phú Yên', 'Quảng Bình', 'Quảng Nam', 'Quảng Ngãi',
        'Quảng Ninh', 'Quảng Trị', 'Sóc Trăng', 'Sơn La','TT Huế',
        'Thanh Hóa', 'Thái Bình', 'Thái Nguyên', 'Tiền Giang', 'Trà Vinh',
        'Tuyên Quang', 'Tây Ninh', 'Vĩnh Phúc', 'Yên Bái', 'Điện Biên',
         'Đà Nẵng', 'Đắk Nông', 'Đắk Lắk', 'Đồng Tháp']



def get_dict_all_city_data():
  cities_data = {}  
  for city in cities:
    city_result = pd.read_excel(data_path+'squeezed_'+city+'.xlsx')  
    # Đoạn này rất quan trọng. Optimize không được đụng vào 36 tháng (2015-2017) để dự báo. 
    # Lúc train và predict thì ok. Lấy từ 1997 -2017. 
    # Function split data sẽ lấy 3 năm cuối làm test data theo config     
    city_result = city_result.loc[city_result['year_month'] < '2018-1-1'] 
    cities_data[city] = city_result
  return cities_data

dict_full_data = get_dict_all_city_data()

In [9]:
def split_data(data, look_back ):
    """Splits data into train and test sets based on args (Configuration class)"""
    train = data[: -args.test_size]    
    test = data[-args.test_size - look_back: ]
    return train, test


In [10]:
def to_supervised(data, d_in=args.look_back, d_out=args.n_predicted_month, features_list=[]):
    """
    Frames time-series as supervised learning dataset.
    
    Args:
      d_in: lookback window
      d_out: number of predicted months
      features_list: list of all features **where last col is the disease incidence**

    Returns:
      Numpy arrays of disease incidence (y) and other predictors (X)
    """
    X, y = list(), list()
    for index, _ in enumerate(data):
        in_end = index + d_in
        out_end = in_end + d_out
        if out_end <= len(data):
            if len(features_list) == 0 :
                X.append(data[index: in_end, :])
            else:
                X.append(data[index: in_end, features_list])
            y.append(data[in_end: out_end, -1])
    return np.array(X), np.array(y)

In [11]:
def select_feature(train, specific_data):
    """Selects args.n_features top features using RFE"""
    train_X, train_y = to_supervised(train, d_in=1, d_out=1)
    train_X, train_y = np.squeeze(train_X), np.squeeze(train_y)
    rfe = RFE(RandomForestRegressor(n_estimators=500, random_state=args.seed), n_features_to_select=args.n_features)
    fit = rfe.fit(train_X, train_y)
    important_features = list()
    print("Important Feature:")
    for i in range(len(fit.support_)):
        if fit.support_[i]:
            important_features.append(i)
            print(specific_data.columns[i])
    return np.array(important_features)

In [12]:
def get_data(train_np, test_np, batch_size, specific_data):
    """
    Returns important feature list and data formatted for input into Pytorch 
    models
    """
    important_features = select_feature(train_np, specific_data)

    train_X, train_y = to_supervised(train_np, features_list=important_features)
    test_X, test_y = to_supervised(test_np, features_list=important_features)
    train_tensor = TensorDataset(torch.from_numpy(train_X), torch.from_numpy(train_y))
    test_tensor = (torch.from_numpy(test_X), torch.from_numpy(test_y))

    train_loader = DataLoader(train_tensor, batch_size=batch_size, shuffle=True)

    return important_features, train_loader, test_tensor

In [13]:
#Define Pytorch LSTM model
class MultiVariateLSTM(nn.Module):
    def __init__(self, n_feature=3, n_layers=2, hidden_size=50):
        super(MultiVariateLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=n_feature, hidden_size=hidden_size, num_layers=n_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, args.n_predicted_month)
        self.loss_fn = nn.MSELoss()
        self.sigma = nn.Parameter(torch.ones(args.n_predicted_month))
    
    def forward(self, X_batch, y_batch=None):
        output, (last_hidden, _) = self.lstm(X_batch)
        last_hidden_vector = output[:, -1, :]
        y_predicted = self.linear(last_hidden_vector)
        if y_batch != None:
            assert y_predicted.size() == y_batch.size()
            loss = self.loss_fn(y_predicted, y_batch)
            loss = 0.5 * loss / self.sigma**2
            loss = loss.sum() + torch.log(1 + self.sigma.prod())
            return y_predicted, loss
            #return y_predicted, self.loss_fn(y_predicted, y_batch)
        else:
            return y_predicted
    
    def predict(self, X):
        X = torch.tensor(X, device=args.device)
        return self.forward(X)

#Define Pytorch LSTM-ATT model
class MultiVariateLSTM_Attention(nn.Module):
    def __init__(self, n_feature=3, n_layers=2, hidden_size=50):
        super(MultiVariateLSTM_Attention, self).__init__()
        self.lstm = nn.LSTM(input_size=n_feature, hidden_size=hidden_size, num_layers=n_layers, batch_first=True)
        self.attention_linear = nn.Linear(hidden_size, hidden_size)
        # self.linear = nn.Linear(hidden_size*2, args.n_predicted_month)
        self.linear = nn.Linear(hidden_size, args.n_predicted_month)
        self.loss_fn = nn.MSELoss()
        self.sigma = nn.Parameter(torch.ones(args.n_predicted_month))
    
    def forward(self, X_batch, y_batch=None):
        output, (last_hidden, _) = self.lstm(X_batch)
        last_hidden_vector = last_hidden[-1]
        remain_hidden_vector = output
        e_t = remain_hidden_vector.bmm(self.attention_linear(last_hidden_vector).unsqueeze(2)).squeeze(-1)
        alpha_t = F.softmax(e_t, dim=1)
        attenion_vector = remain_hidden_vector.transpose(2, 1).bmm(alpha_t.unsqueeze(2)).squeeze(-1)
        # combine_vector = torch.cat((last_hidden_vector, attenion_vector), dim=1)
        # combine_vector = last_hidden_vector + attenion_vector
        y_predicted = self.linear(attenion_vector)
        if y_batch != None:
            assert y_predicted.size() == y_batch.size()
            loss = self.loss_fn(y_predicted, y_batch)
            loss = 0.5 * loss / self.sigma**2
            loss = loss.sum() + torch.log(1 + self.sigma.prod())
            return y_predicted, loss
            # return y_predicted, self.loss_fn(y_predicted, y_batch)
        else:
            return y_predicted
    
    def predict(self, X):
        X = torch.tensor(X, device=args.device)
        return self.forward(X)

# Define Pytorch CNN model
class MultivariateCNN(nn.Module):
    def __init__(self, num_filters=[100, 100, 100], dropout=0.01):
        super(MultivariateCNN, self).__init__()
        self.loss_fn = loss = nn.MSELoss()
        self.filter_sizes = [1, 2, 3]
        self.conv1d_list = nn.ModuleList([nn.Conv1d(args.n_features, num_filters[i], self.filter_sizes[i]) for i in range(len(self.filter_sizes))])
        self.linear = nn.Linear(np.sum(num_filters), args.n_predicted_month)
        self.dropout = nn.Dropout(dropout)
        self.sigma = nn.Parameter(torch.ones(args.n_predicted_month))

    def forward(self, X_batch, y_batch=None):
        X_batch = X_batch.permute(0, 2, 1)  #(batch_size, n_features, n_look_back)
        X_conv_list = [F.relu(conv1d(X_batch)) for conv1d in self.conv1d_list]
        X_pool_list = [F.max_pool1d(x_conv, kernel_size=x_conv.shape[2]) for x_conv in X_conv_list]
        X_fc = torch.cat([x_pool.squeeze(dim=2) for x_pool in X_pool_list], dim=1)
        y_predicted = self.linear(self.dropout(X_fc))
        if y_batch != None:
            assert y_predicted.size() == y_batch.size()
            loss = self.loss_fn(y_predicted, y_batch)
            loss = 0.5 * loss / self.sigma**2
            loss = loss.sum() + torch.log(1 + self.sigma.prod())
            return y_predicted, loss
            # return y_predicted, self.loss_fn(y_predicted, y_batch)
        else:
            return y_predicted

    def predict(self, X):
        X = torch.tensor(X, device=args.device)
        return self.forward(X)

In [14]:
# Create class to train and evaluate models
class Trainer():
    def __init__(self, city, model_type, learning_rate, important_features, train_loader, test_tensor, n_layers=2, hidden_size=128, num_filters=[100, 100, 100], dropout=0.01):
        """
        Initialise trainer, allowing input of LSTM, LSTM-ATT, or CNN 
        hyperparameters. Adam optimiser used for all models.
        """
        self.model_type = model_type    
        self.city = city    
        self.model = self.init_model(model_type, n_layers, hidden_size, num_filters, dropout, city)
        self.model.double().to(args.device)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=learning_rate)
        self.important_features, self.train_loader, self.test_tensor = important_features, train_loader, test_tensor
    
    def init_model(self, model_type, n_layers, hidden_size, num_filters, dropout, city):        
        #Get hyper param by name and set for n_layers, hidden_size, num_filters, dropout and overwrite the input param  
        # n_layers, hidden_size,num_filters, dropout= get_opt_param_by_name( algo = model_type, city = city)
    
        """Initialise a model based on whether LSTM, LSTM-ATT, or CNN is chosen."""
        if model_type.lower() == 'lstm':
            model = MultiVariateLSTM(args.n_features, n_layers, hidden_size)
        elif model_type.lower() == 'lstm_attention':
            model = MultiVariateLSTM_Attention(args.n_features, n_layers, hidden_size)
        elif model_type.lower() == 'cnn':
            model = MultivariateCNN(num_filters, dropout)
        return model

    def step(self, batch):
        self.model.train()
        X_batch, y_batch = tuple(t.to(args.device) for t in batch)
        self.optimizer.zero_grad()
        y_pred, loss = self.model.forward(X_batch, y_batch)
        loss.backward()
        self.optimizer.step()
        return loss.mean().item()

    def validation(self):
        self.model.eval()
        eval_loss = 0.0

        result = {}

        y_true = np.array([])
        y_pred = np.array([])

        X_batch, y_batch = tuple(t.to(args.device) for t in self.test_tensor)
        with torch.no_grad():
            outputs, loss = self.model.forward(X_batch, y_batch)
            eval_loss = loss.mean().item()

        return eval_loss

    def train(self, epochs=20):
        # print('param of model')
        # print('hidensize',self.model.hidden_size)
        # print('nlayer',self.model.n_layers)  
        best_lost = float("inf")
        best_model = None
        for epoch in range(epochs):
            total_loss = 0.0
            for batch in self.train_loader:
                loss = self.step(batch)
                total_loss += loss
            train_loss = total_loss/len(self.train_loader)
            eval_loss = self.validation()
            if eval_loss < best_lost:
                best_lost = eval_loss
                best_model = copy.deepcopy(self.model)
            if (epoch + 1) == epochs or (epoch + 1) in [c + 1 for c in range(epochs) if c % int(epochs/4) == 0]:
                print(f"Epoch: {epoch:2}/{epochs:2} - train_loss: {train_loss:.4f} - test_loss: {eval_loss:4f}")
        self.model = best_model
        self.model.eval()
        return None



    # Lưu model vào trong thư mục models
    def save_model_to(self, path = '', city =''):       
        torch.save(self.model, path)

    def load_model_to(self, path = ''):       
        return torch.load(path)

    def evaluate_model(self, np_data=None, plot=True, scaled=True, city=None, k_steps=None, y_scaler =None):
        assert scaled, "data must be scaled"
        self.model.eval()
        tensor_data = torch.from_numpy(np_data)
        rmse_list = []
        mae_list = [] 
        mape_list = []

        y_predicted_list = []
        y_true_list = []

        for k_steps in range(1, args.n_predicted_month + 1):
            y_predicted = []
            y_true = []
            for index in range(tensor_data.size(0) - args.look_back):
                X = tensor_data[index: index + args.look_back, self.important_features]
                # yhat = self.model.predict(X.unsqueeze(0)).squeeze()

                yhat = self.model.predict(X.unsqueeze(0))
                yhat = yhat.squeeze()

                y_predicted.append(yhat.detach().cpu().numpy()[k_steps - 1])
                y_true.append(tensor_data[index + args.look_back, -1].detach().cpu().numpy())

            y_predicted = y_scaler.inverse_transform(np.array(y_predicted).reshape(-1, 1)).reshape(-1, )
            y_true = y_scaler.inverse_transform(np.array(y_true).reshape(-1, 1)).reshape(-1, )

            

            rmse = mean_squared_error(y_true, y_predicted, squared=False)
            mae = mean_absolute_error(y_true, y_predicted)
            mape = mean_absolute_percentage_error(y_true, y_predicted)

            rmse_list.append(rmse)
            mae_list.append(mae)
            mape_list.append(mape)

#             print('City: '+self.city+'  _algo:'+self.model_type+'  -RMSE: '+str(rmse))          
#             if plot==True:
#               plt.grid(True)
#               plt.plot(y_predicted, label='predicted')
#               plt.plot(y_true, label='actual')
#               plt.title(f"k-steps = {k_steps} - city: "+self.city+'  _algo:'+self.model_type+'  -RMSE: '+str(rmse))
#               plt.legend()
#               plt.show()

            print('City: '+self.city+'  _algo:'+self.model_type+'  -MAE: '+str(mae))          
            if plot==True:
              plt.grid(True)
              plt.plot(y_predicted, label='predicted')
              plt.plot(y_true, label='actual')
              plt.title(f"k-steps = {k_steps} - city: "+self.city+'  _algo:'+self.model_type+'  -MAE: '+str(mae))
              plt.legend()
              plt.show()
    
            y_predicted_list.append(y_predicted)
            y_true_list.append(y_true)

        return y_true_list, y_predicted_list, rmse_list, mae_list, mape_list


In [15]:
def convert_str_num_filter_array(str_filter = ''):  
  arr_filter = str_filter.split(',')  
  int_arr_filter= [int(x) for x in arr_filter]
  return int_arr_filter

# Get Optimized HyperParam

In [16]:
opt_param_cnn = pd.read_excel(prj_path_opt+'/diarrhoea_opt_hyperparam_cnn.xlsx', index_col = 'City')
opt_param_lstm = pd.read_excel(prj_path_opt+'/diarrhoea_opt_hyperparam_lstm.xlsx', index_col = 'City')
opt_param_lstm_att = pd.read_excel(prj_path_opt+'/diarrhoea_opt_hyperparam_lstm_att.xlsx', index_col = 'City')

def get_opt_param_by_name(algo = 'cnn', city = None):
  if algo == 'cnn':
    city_param = opt_param_cnn.loc[city]
    #load  param
    n_layers = ''
    hidden_size = ''
    num_filters = convert_str_num_filter_array(city_param['Num. filters'])
    dropout = float(city_param['Dropout rate'])
    learning_rate = float(city_param['Learning rate'])
    epoch = int(city_param['Epochs'])
    lookback_window = int(city_param['Lookback Window'])
  elif algo == 'lstm':
    city_param = opt_param_lstm.loc[city]
    #load  param
    n_layers = int(city_param['n Layers'])
    hidden_size = int(city_param['Hiden Size'])
    num_filters = ''
    dropout = ''
    learning_rate = float(city_param['Learning rate'])
    epoch = int(city_param['Epochs'])
    lookback_window = int(city_param['Lookback Window'])
  elif algo == 'lstm_attention':
    city_param = opt_param_lstm_att.loc[city]
    #load  param
    n_layers = int(city_param['n Layers'])
    hidden_size = int(city_param['Hiden Size'])
    num_filters = ''
    dropout = ''
    learning_rate = float(city_param['Learning rate'])
    epoch = int(city_param['Epochs'])
    lookback_window = int(city_param['Lookback Window'])
  return n_layers, hidden_size, num_filters, dropout, learning_rate, epoch, lookback_window


In [17]:
# Manual Check param before train
# n_layers, hidden_size, num_filters, dropout, learning_rate, epoch, lookback_window= get_opt_param_by_name( algo = 'cnn', city = 'Cao Bằng')
# lookback_window,epoch, hidden_size, n_layers,learning_rate, num_filters, dropout

# Chạy tay test từng giải thuật

In [18]:
# ################################################
# ## Manual test each city or cities lstm att
# ################################################
# n_LoopEachExpRun = 1
# expRuntime = 0 # basicly just increase each time

# cities = ['BR Vũng Tàu']
# # cities = ['Điện Biên', 'Thái Bình', 'Lào Cai', 'Kon Tum', 'Cao Bằng']

# # An Giang', 'BR Vũng Tàu', 'Bình Phước', 'Bình Thuận', 'Bình Định',
# #        'Bạc Liêu', 'Bắc Cạn', 'Bắc Giang', 'Cao Bằng', 'Cà Mau',
# #        'Cần Thơ', 'Gia Lai', 'Hà Giang', 'Hà Nội', 'Hà Tính', 'Hòa Bình',
# #        'Hưng Yên', 'Hải Dương', 'Hải Phòng', 'Khánh Hòa', 'Kiên Giang',
# #        'Kon Tum', 'Lai Châu', 'Long An', 'Lào Cai', 'Lâm Đồng',
# #        'Lạng Sơn', 'Nam Định', 'Nghệ An', 'Ninh Bình', 'Ninh Thuận',
# #        'Phú Thọ', 'Phú Yên', 'Quảng Bình', 'Quảng Nam', 'Quảng Ngãi',
# #        'Quảng Ninh', 'Quảng Trị', 'Sóc Trăng', 'Sơn La', 'TT Huế',
# #        'Thanh Hóa', 'Thái Bình', 'Thái Nguyên', 'Tiền Giang', 'Trà Vinh',
# #        'Tuyên Quang', 'Tây Ninh', 'Vĩnh Phúc', 'Yên Bái', 'Điện Biên',
# #        'Đà Nẵng', 'Đắc Nông', 'Đắk Lắk', 'Đồng Tháp'


# path = '/content/drive/MyDrive/Colab Notebooks/HealthCare/Source_28_11/results/'
# path_metrix = '/content/drive/MyDrive/Colab Notebooks/HealthCare/Source_28_11/results/metrix/'
# path_models = '/content/drive/MyDrive/Colab Notebooks/HealthCare/Source_28_11/results/models/'
# # provinces = all cities

# l_errCity = {}
# # Với mỗi thành phố ta sẽ chạy n_LoopEachExpRun lần ứng với từng giải thuật.
# # Nếu chỉ chạy 1 lần thì để n_LoopEachExpRun = 0 

# for city in cities:
#   try:

#     total_result = pd.DataFrame() # final result cho mỗi thành phố of all runninng time
#     total_metrix_result = pd.DataFrame() # final matrix result cho mỗi thành phố of all runninng time

#     for interation in range(n_LoopEachExpRun):

#       # Pre-process data
#       specific_data = get_city_data(fix_text(city))   
#       specific_data = impute_missing_value(specific_data)
#       specific_data = convert_to_stationary(specific_data)
#       specific_data.dropna(inplace=True)
#       specific_data_cnn= specific_data_lstm= specific_data_lstm_att = specific_data # lưu cho giải thuật sau.

#       #############################################################
#       # Generate LSTM-ATT model and define province-specific params
#       #############################################################
#       # Get right optimize parame
#       n_layers, hidden_size, num_filters, dropout, learning_rate, epoch, lookback_window = get_opt_param_by_name( algo = 'lstm_attention', city = city)
#       train, test = split_data(specific_data_lstm_att, lookback_window )

#       # Fit data scaler to training data
#       full_scaler = MinMaxScaler().fit(train)
#       y_scaler = MinMaxScaler().fit(train.values[:, -1].reshape(-1, 1))

#       # Scale train and test data
#       train = full_scaler.transform(train)
#       test = full_scaler.transform(test)

      
#       # Get data to run model
#       important_features, train_loader, test_tensor = get_data(train, test, args.batch_size, specific_data_lstm_att)             

     

#       # Overwrite parameters for province-specific diarrhoea prediction
#       args.look_back = lookback_window

#       trainer = Trainer(model_type= 'lstm_attention',
#                         city = city,
#                         learning_rate= learning_rate,
#                         important_features=important_features,
#                         train_loader=train_loader,
#                         test_tensor=test_tensor,
#                         n_layers= n_layers,
#                         hidden_size= hidden_size)

#       # Train model with province-specific epochs
#       trainer.train(epochs=epoch)

#       # Evaluate model  
#       # get predicted value of Test set 
#       y_true_test, y_pred_test, rmse_list, mae_list, mape_list = trainer.evaluate_model(np_data=test, plot= True, scaled=True, city=city, y_scaler= y_scaler) 

#       # get predicted value of Train set 
#       y_true_train, y_pred_train,  _, _, _, = trainer.evaluate_model(np_data=train, plot= True, scaled=True, city=city, y_scaler= y_scaler)                
     
      
#       # #############################################################
#       # Generate LSTM model and define province-specific params
#       # Get right optimize parame
#       n_layers, hidden_size, num_filters, dropout, learning_rate, epoch, lookback_window = get_opt_param_by_name( algo = 'lstm', city = city)
#       train, test = split_data(specific_data_lstm_att, lookback_window )

#       # Fit data scaler to training data
#       full_scaler = MinMaxScaler().fit(train)
#       y_scaler = MinMaxScaler().fit(train.values[:, -1].reshape(-1, 1))

#       # Scale train and test data
#       train = full_scaler.transform(train)
#       test = full_scaler.transform(test)

      
#       # Get data to run model
#       important_features, train_loader, test_tensor = get_data(train, test, args.batch_size, specific_data_lstm_att)  

#       # Overwrite parameters for province-specific diarrhoea prediction
#       args.look_back = lookback_window

#       trainer = Trainer(model_type='lstm', 
#                         city = city,
#                         learning_rate=learning_rate,
#                         important_features=important_features,
#                         train_loader=train_loader,
#                         test_tensor=test_tensor,
#                         n_layers= n_layers,
#                         hidden_size= hidden_size)   
    
#       # Train model with province-specific epochs
#       print('epoc',epoch)
#       trainer.train(epochs=epoch)

#       # Evaluate model  
#       # get predicted value of Test set 
#       y_true_test, y_pred_test, rmse_list, mae_list, mape_list = trainer.evaluate_model(np_data=test, plot= True, scaled=True, city=city, y_scaler= y_scaler) 

#       # get predicted value of Train set 
#       y_true_train, y_pred_train,  _, _, _, = trainer.evaluate_model(np_data=train, plot= True, scaled=True, city=city, y_scaler= y_scaler) 
#   except Exception as e:
#     l_errCity[city] = e
#     break    

# print ('danh sách error city: ')
# print(l_errCity)

In [19]:
# Plot kết quả
# Ntest = y_true_test[0].size # 33
# Loockback_window = lookback_window #6
# mix_train_test_result = pd.DataFrame()
# mix_train_test_result['Observed']= specific_data['Diarrhoea_rates'] #239
# mix_train_test_result = mix_train_test_result.reset_index()
# mix_train_test_result['Train Prediction'] = None
# mix_train_test_result['Test Prediction'] = None
# mix_train_test_result['Train Prediction'][Loockback_window:Loockback_window+y_pred_train[0].size] = y_pred_train[0]
# mix_train_test_result['Test Prediction'].iloc[-(y_pred_test[0].size):] = y_pred_test[0]


In [20]:
# cols = ['Observed',
#         'Train Prediction',
#         'Test Prediction']      

# mix_train_test_result[cols].plot(figsize=(25, 5),grid = True);

In [21]:
# Loockback_window
# mix_train_test_result['Observed'].size
# y_pred_train[0].size
# y_pred_test[0].size
# train1['Diarrhoea_rates'].size
# test1['Diarrhoea_rates'].size
# y_true_test[0].size

# Mix Result và Lưu vào folder tương ứng theo từng algo


In [22]:
def mix_results(city, algo, specific_data, y_pred_test, y_pred_train, lookback_window, iteration, rmse, mae, mape):
  new_cols = ["City","Algorithm","Run Seq","Observed","Train Prediction","Test Prediction","rmse","mae","mape"]
  mix_train_test_result = pd.DataFrame()
  mix_train_test_result['Observed']= specific_data['Diarrhoea_rates'] 
  mix_train_test_result = mix_train_test_result.reset_index(drop=True)  
  mix_train_test_result['City'] = city
  mix_train_test_result['Algorithm'] = algo
  mix_train_test_result['Run Seq'] = iteration 
  mix_train_test_result['Train Prediction'] = None
  mix_train_test_result['Test Prediction'] = None
  mix_train_test_result['Train Prediction'][lookback_window:lookback_window+y_pred_train[0].size] = y_pred_train[0]
  mix_train_test_result['Test Prediction'].iloc[-(y_pred_test[0].size):] = y_pred_test[0]
  mix_train_test_result['rmse'] = rmse
  mix_train_test_result['mae'] = mae
  mix_train_test_result['mape'] = mape
  mix_train_test_result= mix_train_test_result.reindex(columns=new_cols)
  return mix_train_test_result

In [23]:
def save_results(all_mix_train_test_result, algo,city):  
  if algo == 'cnn':    
    all_mix_train_test_result.to_excel(path_all_mix_cnn+'mix_train_test_data_'+city+'_'+algo+'.xlsx', index = False)
  elif algo == 'lstm':
    all_mix_train_test_result.to_excel(path_all_mix_lstm+'mix_train_test_data_'+city+'_'+algo+'.xlsx', index = False)
  elif algo == 'lstm_att':
    all_mix_train_test_result.to_excel(path_all_mix_lstm_att+'mix_train_test_data_'+city+'_'+algo+'.xlsx', index = False)

# Main Run

In [24]:
dt_started = datetime.now()
################################################
## Main run all city for 3 Algorithm
## Mỗi city sẽ cho chạy N lần 3 bộ giải thuật.
## Output sẽ là mỗi thành phố 3 dạng file: 1 file bao gồm predict 36 tháng + metrics, 1 file metric, 3 file predict cả train và test + metrics (bỏ vào mỗi folder).
################################################
n_LoopEachExpRun = 5
expRuntime = 0 # basicly just increase each time

cities = [
        'An Giang', 'BR Vũng Tàu', 'Bình Phước', 'Bình Thuận', 'Bình Định',
        'Bạc Liêu', 'Bắc Kạn', 'Bắc Giang', 'Cao Bằng', 'Cà Mau',
#         'Cần Thơ', 'Gia Lai', 'Hà Giang', 'Hà Nội', 'Hà Tĩnh', 'Hòa Bình',
#         'Hưng Yên', 'Hải Dương', 'Hải Phòng', 'Khánh Hòa', 'Kiên Giang',
#         'Kon Tum', 'Lai Châu', 'Long An', 'Lào Cai', 'Lâm Đồng',
#         'Lạng Sơn', 'Nam Định', 'Nghệ An', 'Ninh Bình', 'Ninh Thuận',
#         'Phú Thọ', 'Phú Yên', 'Quảng Bình', 'Quảng Nam', 'Quảng Ngãi',
#         'Quảng Ninh', 'Quảng Trị', 'Sóc Trăng', 'Sơn La', 'TT Huế',
#         'Thanh Hóa', 'Thái Bình', 'Thái Nguyên', 'Tiền Giang', 'Trà Vinh',
#         'Tuyên Quang', 'Tây Ninh', 'Vĩnh Phúc', 'Yên Bái', 'Điện Biên',
#          'Đà Nẵng', 'Đắk Nông', 'Đắk Lắk', 'Đồng Tháp'
         ]


l_errCity = {} # có lỗi sẽ lưu vào đây, kiểm tra ngay cell sau

for city in cities:
  # Lưu kết quả full cho từng thành phố
  mix_train_test_result_cnn=pd.DataFrame()
  mix_train_test_result_lstm=pd.DataFrame()
  mix_train_test_result_lstm_att=pd.DataFrame()
  try:

    total_result = pd.DataFrame() # final result cho mỗi thành phố of all runninng time
    total_metrix_result = pd.DataFrame() # final matrix result cho mỗi thành phố of all runninng time
    
    for interation in range(n_LoopEachExpRun):
      
      # Pre-process data
      specific_data = get_city_data(fix_text(city))   
      specific_data = impute_missing_value(specific_data)
      specific_data = convert_to_stationary(specific_data)
      specific_data.dropna(inplace=True)

      specific_data_cnn  = specific_data
      specific_data_lstm = specific_data
      specific_data_lstm_att = specific_data
      ########################################################
      # Generate CNN model and define province-specific params
      ########################################################
      # Get right optimize parame
      n_layers, hidden_size, num_filters, dropout, learning_rate, epoch, lookback_window = get_opt_param_by_name( algo = 'cnn', city = city)

      train, test = split_data(specific_data_cnn, lookback_window)

      # Fit data scaler to training data
      full_scaler = MinMaxScaler().fit(train)
      y_scaler = MinMaxScaler().fit(train.values[:, -1].reshape(-1, 1))

      # Scale train and test data
      train = full_scaler.transform(train)
      test = full_scaler.transform(test)

      
      # Get data to run model
      important_features, train_loader, test_tensor = get_data(train, test, args.batch_size, specific_data_cnn)    
      

      # Overwrite parameters for province-specific diarrhoea prediction
      args.look_back = lookback_window
      trainer = Trainer(
                        model_type='cnn',  
                        city = city,                    
                        learning_rate=learning_rate,
                        important_features=important_features,
                        train_loader=train_loader,
                        test_tensor=test_tensor,
                        num_filters= num_filters, 
                        dropout=dropout )
    
      # Train model with province-specific epochs
      trainer.train(epochs=epoch)           
      

      # Evaluate test set   
      y_true_test, y_pred_test, rmse_list, mae_list, mape_list = trainer.evaluate_model(np_data=test, plot= False, scaled=True, city=city, y_scaler= y_scaler)
      
      # Evaluate train  set   
      y_true_train, y_pred_train,  _, _, _, = trainer.evaluate_model(np_data=train, plot= False, scaled=True, city=city, y_scaler= y_scaler)
      
      # Lưu tất cả kết quả predicted train và predicted test
      mix_train_test_result_cnn = mix_train_test_result_cnn.append(mix_results(city, 'cnn', specific_data_cnn, y_pred_test, y_pred_train,lookback_window,interation, rmse_list[0],mae_list[0],mape_list[0])) 

      cnn_reslt = pd.DataFrame({
                              'City': city,
                              'Alg_name': 'cnn',
                              'Run Seq': interation,
                              'Observed': y_true_test[0], 
                              '1-month': y_pred_test[0], 
                              '2-months': y_pred_test[1],
                              '3-months': y_pred_test[2],
                              'RMSE_1-month': rmse_list[0],
                              'RMSE_2-month': rmse_list[1],
                              'RMSE_3-month': rmse_list[2],
                              'MAE_1-month': mae_list[0],
                              'MAE_2-month': mae_list[1],
                              'MAE_3-month': mae_list[2],
                              'MAPE_1-month': mape_list[0],
                              'MAPE_2-month': mape_list[1],
                              'MAPE_3-month': mape_list[2]})
      
      cnn_metrix_reslt = pd.DataFrame({
                              'City': city,
                              'Alg_name': 'cnn',
                              'Run Seq': interation,                          
                              'RMSE_1-month': rmse_list[0],
                              'RMSE_2-month': rmse_list[1],
                              'RMSE_3-month': rmse_list[2],
                              'MAE_1-month': mae_list[0],
                              'MAE_2-month': mae_list[1],
                              'MAE_3-month': mae_list[2],
                              'MAPE_1-month': mape_list[0],
                              'MAPE_2-month': mape_list[1],
                              'MAPE_3-month': [mape_list[2]]}) # do có 1 dòng nên pandas cần index = 0
      

      #############################################################
      # Generate LSTM-ATT model and define province-specific params
      #############################################################
      # Get right optimize parame
      n_layers, hidden_size, num_filters, dropout, learning_rate, epoch, lookback_window = get_opt_param_by_name( algo = 'lstm_attention', city = city)

      train, test = split_data(specific_data_lstm_att, lookback_window)

      # Fit data scaler to training data
      full_scaler = MinMaxScaler().fit(train)
      y_scaler = MinMaxScaler().fit(train.values[:, -1].reshape(-1, 1))

      # Scale train and test data
      train = full_scaler.transform(train)
      test = full_scaler.transform(test)

      
      # Get data to run model
      important_features, train_loader, test_tensor = get_data(train, test, args.batch_size, specific_data_lstm_att) 

      # Overwrite parameters for province-specific diarrhoea prediction
      args.look_back = lookback_window

      trainer = Trainer(model_type= 'lstm_attention',
                        city = city,
                        learning_rate= learning_rate,
                        important_features=important_features,
                        train_loader=train_loader,
                        test_tensor=test_tensor,
                        n_layers= n_layers,
                        hidden_size=hidden_size)

      # Train model with province-specific epochs
      trainer.train(epochs=epoch)
      
      # Evaluate test set   
      y_true_test, y_pred_test, rmse_list, mae_list, mape_list = trainer.evaluate_model(np_data=test, plot= False, scaled=True, city=city, y_scaler= y_scaler)
      
      # Evaluate train  set   
      y_true_train, y_pred_train,  _, _, _, = trainer.evaluate_model(np_data=train, plot= False, scaled=True, city=city, y_scaler= y_scaler)
      
      # Lưu tất cả kết quả predicted train và predicted test
      mix_train_test_result_lstm_att = mix_train_test_result_lstm_att.append(mix_results(city,'lstm_att', specific_data_lstm_att, y_pred_test, y_pred_train,lookback_window,interation, rmse_list[0],mae_list[0],mape_list[0])) 


      lstm_attention_reslt = pd.DataFrame({
                              'City': city,
                              'Alg_name': 'lstm_attention',
                              'Run Seq': interation,
                              'Observed': y_true_test[0], 
                              '1-month': y_pred_test[0], 
                              '2-months': y_pred_test[1],
                              '3-months': y_pred_test[2],
                              'RMSE_1-month': rmse_list[0],
                              'RMSE_2-month': rmse_list[1],
                              'RMSE_3-month': rmse_list[2],
                              'MAE_1-month': mae_list[0],
                              'MAE_2-month': mae_list[1],
                              'MAE_3-month': mae_list[2],
                              'MAPE_1-month': mape_list[0],
                              'MAPE_2-month': mape_list[1],
                              'MAPE_3-month': mape_list[2]})
      
      lstm_attention_metrix_reslt = pd.DataFrame({
                              'City': city,
                              'Alg_name': 'lstm_attention',
                              'Run Seq': interation,                          
                              'RMSE_1-month': rmse_list[0],
                              'RMSE_2-month': rmse_list[1],
                              'RMSE_3-month': rmse_list[2],
                              'MAE_1-month': mae_list[0],
                              'MAE_2-month': mae_list[1],
                              'MAE_3-month': mae_list[2],
                              'MAPE_1-month': mape_list[0],
                              'MAPE_2-month': mape_list[1],
                              'MAPE_3-month': mape_list[2]}, index=[0]) # do có 1 dòng nên pandas cần index = 0


      # #############################################################
      # Generate LSTM model and define province-specific params
      n_layers, hidden_size, num_filters, dropout, learning_rate, epoch, lookback_window = get_opt_param_by_name( algo = 'lstm', city = city)

      train, test = split_data(specific_data_lstm, lookback_window)

      # Fit data scaler to training data
      full_scaler = MinMaxScaler().fit(train)
      y_scaler = MinMaxScaler().fit(train.values[:, -1].reshape(-1, 1))

      # Scale train and test data
      train = full_scaler.transform(train)
      test = full_scaler.transform(test)

      
      # Get data to run model
      important_features, train_loader, test_tensor = get_data(train, test, args.batch_size, specific_data_lstm) 

      # Overwrite parameters for province-specific diarrhoea prediction
      args.look_back = lookback_window

      trainer = Trainer(model_type='lstm', 
                        city = city,
                        learning_rate=learning_rate,
                        important_features=important_features,
                        train_loader=train_loader,
                        test_tensor=test_tensor,
                        n_layers=n_layers,
                        hidden_size=hidden_size)   
    
      # Train model with province-specific epochs
      trainer.train(epochs=epoch)

      # Evaluate test set   
      y_true_test, y_pred_test, rmse_list, mae_list, mape_list = trainer.evaluate_model(np_data=test, plot= False, scaled=True, city=city, y_scaler= y_scaler)
      
      # Evaluate train  set   
      y_true_train, y_pred_train,  _, _, _, = trainer.evaluate_model(np_data=train, plot= False, scaled=True, city=city, y_scaler= y_scaler)
      
      # Lưu tất cả kết quả predicted train và predicted test, metrics all
      mix_train_test_result_lstm = mix_train_test_result_lstm.append(mix_results(city, 'lstm', specific_data_lstm, y_pred_test, y_pred_train,lookback_window,interation, rmse_list[0],mae_list[0],mape_list[0])) 

      # Lưu full kết quả predict và metrics      
      lstm_reslt = pd.DataFrame({
                              'City': city,
                              'Alg_name': 'lstm',
                              'Run Seq': interation,
                              'Observed': y_true_test[0], 
                              '1-month': y_pred_test[0], 
                              '2-months': y_pred_test[1],
                              '3-months': y_pred_test[2],
                              'RMSE_1-month': rmse_list[0],
                              'RMSE_2-month': rmse_list[1],
                              'RMSE_3-month': rmse_list[2],
                              'MAE_1-month': mae_list[0],
                              'MAE_2-month': mae_list[1],
                              'MAE_3-month': mae_list[2],
                              'MAPE_1-month': mape_list[0],
                              'MAPE_2-month': mape_list[1],
                              'MAPE_3-month': mape_list[2]})

      # Lưu full kết quả metrics thôi
      lstm_metrix_reslt = pd.DataFrame({
                              'City': city,
                              'Alg_name': 'lstm',
                              'Run Seq': interation,                          
                              'RMSE_1-month': rmse_list[0],
                              'RMSE_2-month': rmse_list[1],
                              'RMSE_3-month': rmse_list[2],
                              'MAE_1-month': mae_list[0],
                              'MAE_2-month': mae_list[1],
                              'MAE_3-month': mae_list[2],
                              'MAPE_1-month': mape_list[0],
                              'MAPE_2-month': mape_list[1],
                              'MAPE_3-month': mape_list[2]}, index=[0]) # do có 1 dòng nên pandas cần index = 0
      
      

      total_result = total_result.append(pd.DataFrame(cnn_reslt))
      total_result = total_result.append(pd.DataFrame(lstm_attention_reslt))
      total_result = total_result.append(pd.DataFrame(lstm_reslt))

      total_metrix_result = total_metrix_result.append(pd.DataFrame(cnn_metrix_reslt))
      total_metrix_result = total_metrix_result.append(pd.DataFrame(lstm_attention_metrix_reslt))
      total_metrix_result = total_metrix_result.append(pd.DataFrame(lstm_metrix_reslt))
      
    # Nếu tron n lần chạy 3 bộ giải thuật không có lỗi gì thì:
      # Lưu xuống file excel ứng với thành phố và 3 giải thuật deep learning tương ứng
      # Lưu xuống mix predicted train và test data ứng với từng thành phố/ từng giải thuật
    total_result.to_excel(path+'diarrhoea_'+city+'_'+str(expRuntime)+'.xlsx')  
    total_metrix_result.to_excel(path_metrix+'diarrhoea_metrix_'+city+'_'+str(expRuntime)+'.xlsx')

    # Lưu xuống mix predicted train và test data ứng với từng thành phố/ từng giải thuật
    save_results(mix_train_test_result_cnn,'cnn', city)
    save_results(mix_train_test_result_lstm,'lstm', city)
    save_results(mix_train_test_result_lstm_att,'lstm_att', city)
  except Exception as e:
    l_errCity[city] = e
    break    


dt_ended = datetime.now()
print('kết thúc train trong:', round((dt_ended - dt_started).total_seconds()/60))
print(l_errCity)

Important Feature:
Total_Rainfall
Average_temperature
Diarrhoea_rates
Epoch:  0/150 - train_loss: 0.7045 - test_loss: 0.570026
Epoch: 37/150 - train_loss: 0.2128 - test_loss: 0.120011
Epoch: 74/150 - train_loss: 0.2237 - test_loss: 0.138816
Epoch: 111/150 - train_loss: 0.2162 - test_loss: 0.119565
Epoch: 148/150 - train_loss: 0.2203 - test_loss: 0.101990
Epoch: 149/150 - train_loss: 0.2070 - test_loss: 0.108139
City: An Giang  _algo:cnn  -MAE: 16.779600891661943
City: An Giang  _algo:cnn  -MAE: 15.885452339947888
City: An Giang  _algo:cnn  -MAE: 15.034881584596699
City: An Giang  _algo:cnn  -MAE: 31.516759752739752
City: An Giang  _algo:cnn  -MAE: 32.17326131891442
City: An Giang  _algo:cnn  -MAE: 32.05232128866981


<ipython-input-22-d792b7109edd>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mix_train_test_result['Train Prediction'][lookback_window:lookback_window+y_pred_train[0].size] = y_pred_train[0]
<ipython-input-22-d792b7109edd>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mix_train_test_result['Test Prediction'].iloc[-(y_pred_test[0].size):] = y_pred_test[0]
<ipython-input-24-ac7150b26928>:92: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mix_train_test_result_cnn = mix_train_test_result_cnn.append(mix_results(city, 'cnn', spe

Important Feature:
Total_Rainfall
Average_temperature
Diarrhoea_rates
Epoch:  0/190 - train_loss: 0.9091 - test_loss: 0.758792
Epoch: 47/190 - train_loss: 0.1977 - test_loss: 0.095863
Epoch: 94/190 - train_loss: 0.1916 - test_loss: 0.097273
Epoch: 141/190 - train_loss: 0.1871 - test_loss: 0.089692
Epoch: 188/190 - train_loss: 0.1832 - test_loss: 0.085756
Epoch: 189/190 - train_loss: 0.2114 - test_loss: 0.094473
City: An Giang  _algo:lstm_attention  -MAE: 6.3025079253619545
City: An Giang  _algo:lstm_attention  -MAE: 5.653171305229489
City: An Giang  _algo:lstm_attention  -MAE: 6.770696667749323
City: An Giang  _algo:lstm_attention  -MAE: 31.33097038418976
City: An Giang  _algo:lstm_attention  -MAE: 34.76341571595404
City: An Giang  _algo:lstm_attention  -MAE: 36.333329101794156


<ipython-input-22-d792b7109edd>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mix_train_test_result['Train Prediction'][lookback_window:lookback_window+y_pred_train[0].size] = y_pred_train[0]
<ipython-input-22-d792b7109edd>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mix_train_test_result['Test Prediction'].iloc[-(y_pred_test[0].size):] = y_pred_test[0]
<ipython-input-24-ac7150b26928>:169: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mix_train_test_result_lstm_att = mix_train_test_result_lstm_att.append(mix_results(city,

Important Feature:
Total_Rainfall
Average_temperature
Diarrhoea_rates
Epoch:  0/330 - train_loss: 0.6591 - test_loss: 0.537464
Epoch: 82/330 - train_loss: 0.1718 - test_loss: 0.087300
Epoch: 164/330 - train_loss: 0.1739 - test_loss: 0.111475
Epoch: 246/330 - train_loss: 0.1806 - test_loss: 0.098740
Epoch: 328/330 - train_loss: 0.1663 - test_loss: 0.081741
Epoch: 329/330 - train_loss: 0.1800 - test_loss: 0.079286
City: An Giang  _algo:lstm  -MAE: 18.120670016025958
City: An Giang  _algo:lstm  -MAE: 17.11410095710952
City: An Giang  _algo:lstm  -MAE: 19.140710034578742
City: An Giang  _algo:lstm  -MAE: 34.749322931288226
City: An Giang  _algo:lstm  -MAE: 37.5420854609324
City: An Giang  _algo:lstm  -MAE: 37.49966382646895


...

Important Feature:
Total_Evaporation
Max_Daily_Rainfall
Diarrhoea_rates
Epoch:  0/340 - train_loss: 0.9082 - test_loss: 0.750651
Epoch: 85/340 - train_loss: 0.3524 - test_loss: 0.327882
Epoch: 170/340 - train_loss: 0.1928 - test_loss: 0.140609
Epoch: 255/340 - train_loss: 0.1594 - test_loss: 0.081329
Epoch: 339/340 - train_loss: 0.1519 - test_loss: 0.074825
City: Cà Mau  _algo:lstm_attention  -MAE: 3.5275442603029807
City: Cà Mau  _algo:lstm_attention  -MAE: 3.4771491758765225
City: Cà Mau  _algo:lstm_attention  -MAE: 6.107656316772436
City: Cà Mau  _algo:lstm_attention  -MAE: 12.00450793059415
City: Cà Mau  _algo:lstm_attention  -MAE: 11.996269288241068
City: Cà Mau  _algo:lstm_attention  -MAE: 12.253147988872094


<ipython-input-22-d792b7109edd>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mix_train_test_result['Train Prediction'][lookback_window:lookback_window+y_pred_train[0].size] = y_pred_train[0]
<ipython-input-22-d792b7109edd>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mix_train_test_result['Test Prediction'].iloc[-(y_pred_test[0].size):] = y_pred_test[0]
<ipython-input-24-ac7150b26928>:169: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mix_train_test_result_lstm_att = mix_train_test_result_lstm_att.append(mix_results(city,

Important Feature:
Total_Evaporation
Max_Daily_Rainfall
Diarrhoea_rates
Epoch:  0/130 - train_loss: 0.7948 - test_loss: 0.691085
Epoch: 32/130 - train_loss: 0.5690 - test_loss: 0.553526
Epoch: 64/130 - train_loss: 0.4455 - test_loss: 0.423999
Epoch: 96/130 - train_loss: 0.3450 - test_loss: 0.319960
Epoch: 128/130 - train_loss: 0.2656 - test_loss: 0.240388
Epoch: 129/130 - train_loss: 0.2629 - test_loss: 0.230687
City: Cà Mau  _algo:lstm  -MAE: 4.350963527045301
City: Cà Mau  _algo:lstm  -MAE: 4.000689880532102
City: Cà Mau  _algo:lstm  -MAE: 5.429082076676289
City: Cà Mau  _algo:lstm  -MAE: 12.022063285541353
City: Cà Mau  _algo:lstm  -MAE: 11.771931957755884
City: Cà Mau  _algo:lstm  -MAE: 11.965346681227333


<ipython-input-22-d792b7109edd>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mix_train_test_result['Train Prediction'][lookback_window:lookback_window+y_pred_train[0].size] = y_pred_train[0]
<ipython-input-22-d792b7109edd>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mix_train_test_result['Test Prediction'].iloc[-(y_pred_test[0].size):] = y_pred_test[0]
<ipython-input-24-ac7150b26928>:245: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mix_train_test_result_lstm = mix_train_test_result_lstm.append(mix_results(city, 'lstm',

kết thúc train trong: 136
{}


In [25]:
import requests

def send_to_telegram(message):

    apiToken = '5908735099:AAGVSLrW62aXPBP-GrMvxoVgMsuJxXJpP1Q'
    chatID = '@ptn_announcement'
    apiURL = f'https://api.telegram.org/bot{apiToken}/sendMessage'

    try:
        response = requests.post(apiURL, json={'chat_id': chatID, 'text': message})
        print(response.text)
    except Exception as e:
        print(e)

send_to_telegram("train + predict xong")

{"ok":true,"result":{"message_id":451,"sender_chat":{"id":-1001712314864,"title":"PTN_Announcement","username":"ptn_announcement","type":"channel"},"chat":{"id":-1001712314864,"title":"PTN_Announcement","username":"ptn_announcement","type":"channel"},"date":1676136906,"text":"train + predict xong"}}
